<a href="https://colab.research.google.com/github/Harshpatel44/MNIST-dataset/blob/master/MNIST_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
a = [1,2,3,4]
b = [5,6,3,2]
count=0
print(set(a) & set(b))


{2, 3}


Second version of the code where there is only 1 for loop

In [10]:
import numpy as np
import time
import threading
import pickle
import matplotlib.pyplot as plt

#from google.colab import files
#uploaded = files.upload()
downloaded = drive.CreateFile({'id':'1PzGPanqXSdwsXebqaKbO7uaNQidNjsOK'}) 
downloaded.GetContentFile('mnist_train.csv')  

downloaded2 = drive.CreateFile({'id':'1i4ezuhBEc_VBEWnJ9WS7wQLGP-n-wYOL'}) 
downloaded.GetContentFile('mnist_test.csv')

def data_creation():
    X1, X2 = np.mgrid[-1.5:1.5:0.1, -1.5:1.5:0.1]
    X = np.array([X1.flatten(), X2.flatten()]).T
    y = np.zeros((X.shape[0]), dtype='int32')

    # generating classes
    for i in range(0, X.shape[0]):
        if (np.linalg.norm([X[i][0], X[i][1]], 2) <= 1):
            y[i] = 1
        else:
            y[i] = 2

    #plotting
    plt.scatter(X[:, 0], X[:, 1], c=y)
    plt.show()
    return X,y

#X,y=data_creation()

def data_creation():
    X1, X2 = np.mgrid[-1.5:1.5:0.1, -1.5:1.5:0.1]
    X = np.array([X1.flatten(), X2.flatten()]).T
    y = np.zeros((X.shape[0]), dtype='int32')

    # generating classes
    for i in range(0, X.shape[0]):
        if (np.linalg.norm([X[i][0], X[i][1]], 2) <= 1):
            y[i] = 1
        else:
            y[i] = 2
    return X,y

X,y=data_creation()

class KNN:
    def __init__(self, k, num_classes):
        self.num_classes = num_classes
        self.k = k
        self.parameters = None

    def train(self, training_data, training_labels):
        """
        Train the classifier using the training data and training labels.
        Store whatever you think is neccessary in self.parameters.
        """

        self.X=np.array(training_data,dtype='float16')
        self.y= training_labels
        self.parameters = None
        pass

    def process(self,data,eucli,count):
        # creating an array to store whole matrix, first value is inserted, which will be deleted later

        e = np.linalg.norm(eucli[:, :-1] - data, 2, axis=1)
        e = e.reshape(e.shape[0], 1)

        eucli = np.append(eucli, e, axis=1)

        # sorting with respect to euclidian distances and taking only kth neighbours
        eucli = eucli[eucli[:, -1].argsort()][:self.k]

        # finding number of values with class_1 and class_2
        unique, counts = np.unique(eucli[:, -2], return_counts=True)
        count_dict = dict(zip(unique, counts))
        #print('number',int(max(count_dict, key=count_dict.get)))
        self.predic_list.append([int(max(count_dict, key=count_dict.get)),count])
        return

    def predict(self, data):
        self.predic_list=[]
        """
        Make use of the trained parameters to predict the class.
        Try using np.lingalg.norm() to compute distances.
        Input should be a LIST of d-dimensional points.
        The return value should be a LIST of scalars between 1 and num_classes,
        indicating for each input point how it is classified.
        """
        threads=[]
        eucli = np.array((self.X), dtype="float16")
        eucli = np.append(eucli, self.y.reshape(self.y.shape[0], 1), axis=1)

        count=0
        for k in np.array(data,dtype='int16'):
            count+=1
            t=threading.Thread(target=KNN.process ,args=(self,k,eucli,count))
            threads.append(t)
            t.start()

        for t in threads:
            t.join()
        return self.predic_list

    def accuracy(self, data, labels):
        """
        Returns the proportion of correct classifications computed over data.
        """
        count=0
        predic_list=self.predict(data)
        predic_list.sort(key = lambda x: x[1])
        for i,j in zip(predic_list,labels):
            if(i[0]==j):
                count+=1
        return count/len(labels)


import csv

#reading the file
f1 = open ('mnist_train.csv','r')
train=list(csv.reader(f1,delimiter=','))
X=np.array(train,dtype="int16")
y = X[:,0]

f2 = open ('mnist_test.csv','r')
test=list(csv.reader(f2,delimiter=','))
val_X=np.array(test,dtype="int16")


# run method creates KNN class, train and predict the data.
def run():
    start = time.time()
    classifier=KNN_implementation.KNN(k=1,num_classes=10)
    classifier.train(X[:,1:],y[:])
    #predictions=classifier.predict(X[2600:2620,1:])
    #print(predictions)
    #print(y[2600:2620])
    acc=classifier.accuracy(val_X[:,1:],y[:])
    print(acc)
    end = time.time()
    print(end-start)
#run()

def k_find():
    list=[]
    for i in range(1,2):
        start = time.time()
        classifier=KNN(k=i,num_classes=10)
        classifier.train(X[1:10000,1:],y[1:10000])
        #print(X[3:7,1:])
        #predictions=classifier.predict(X[501:510,1:])
        #print(predictions)
        #print(y[501:510])
        acc=classifier.accuracy(X[10001:12000,1:],y[10001:12000])
        end = time.time()
        print(acc,end-start)
        list.append([i,acc,end-start])
    print(list)

    f = open ('k_values.pickle','wb')
    pickle.dump(list,f)
    f.close()

k_find()

0.9484742371185593 37.52729272842407
[[1, 0.9484742371185593, 37.52729272842407]]


using SKlearn library to compare the output and accuracy


In [3]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

import csv
import time

downloaded = drive.CreateFile({'id':'1PzGPanqXSdwsXebqaKbO7uaNQidNjsOK'}) 
downloaded.GetContentFile('mnist_train.csv')  

f = open ('mnist_train.csv','r')
wines=list(csv.reader(f,delimiter=','))
X=np.array(wines,dtype="int16")
y = X[:,0]

c_time=time.time()
neigh = KNeighborsClassifier(n_neighbors = 1)
neigh.fit(X[1:10000,1:],y[1:10000])
#neigh.predict(X[501:510,1:])

x=neigh.score(X[10001:12000,1:],y[10001:12000])
f_time=time.time()
print(x,f_time-c_time)

0.9484742371185593 30.261850118637085


In [0]:
import numpy as np
import threading
import matplotlib.pyplot as plt

def data_creation():
    X1, X2 = np.mgrid[-1.5:1.5:0.1, -1.5:1.5:0.1]
    X = np.array([X1.flatten(), X2.flatten()]).T
    y = np.zeros((X.shape[0]), dtype='int32')

    # generating classes
    for i in range(0, X.shape[0]):
        if (np.linalg.norm([X[i][0], X[i][1]], 2) <= 1):
            y[i] = 1
        else:
            y[i] = 2

    #plotting
    #plt.scatter(X[:, 0], X[:, 1], c=y)
    #plt.show()
    return X,y

X,y=data_creation()

class KNN:
    def __init__(self, k, num_classes):
        self.num_classes = num_classes
        self.k = k
        self.predic_list = []
        self.parameters = None

    def train(self, training_data, training_labels):
        """
        Train the classifier using the training data and training labels.
        Store whatever you think is neccessary in self.parameters.
        """

        self.X=np.array(training_data,dtype='float64')
        self.y= training_labels
        self.parameters = None
        pass

    def process(self,data):
        # creating an array to store whole matrix, first value is inserted, which will be deleted later
        eucli = np.array((self.X), dtype="float16")

        # self.y=self.y.reshape(self.y.shape[0],1)

        eucli = np.append(eucli, self.y.reshape(self.y.shape[0], 1), axis=1)

        e = np.linalg.norm(eucli[:, :-1] - data, 2, axis=1)
        e = e.reshape(e.shape[0], 1)

        eucli = np.append(eucli, e, axis=1)

        # removing the first value, inserted during initialization

        # sorting with respect to euclidian distances and taking only kth neighbours
        eucli = eucli[eucli[:, -1].argsort()][:self.k]

        # finding number of values with class_1 and class_2
        unique, counts = np.unique(eucli[:, -2], return_counts=True)
        count_dict = dict(zip(unique, counts))
        self.predic_list.append(int(max(count_dict, key=count_dict.get)))
        return
    def predict(self, data):
        """
        Make use of the trained parameters to predict the class.
        Try using np.lingalg.norm() to compute distances.
        Input should be a LIST of d-dimensional points.
        The return value should be a LIST of scalars between 1 and num_classes,
        indicating for each input point how it is classified.
        """
        threads=[]
        for k in np.array(data,dtype='int16'):
            t=threading.Thread(target=KNN.process ,args=(self,k,))
            threads.append(t)
            t.start()

        for t in threads:
            t.join()

        print('ended')
        return self.predic_list

    def accuracy(self, data, labels):
        """
        Returns the proportion of correct classifications computed over data.
        """
        predic_list=self.predict(data)
        count=0

        for i,j in zip(predic_list,labels):
            if(i==j):
                count+=1
        return count/len(labels)

import csv
import numpy as np

f = open ('mnist_train.csv','r')
wines=list(csv.reader(f,delimiter=','))
X=np.array(wines,dtype="int16")
y = X[:,0]
#print(y[0:10])

c_time=time.time()
classifier=KNN(k=5,num_classes=10)
classifier.train(X[1:1000,1:],y[1:1000])
#predictions=classifier.predict(X[0:10,1:])
#print(predictions)
#print(X[50001:50010],y[50001:50010])

acc=classifier.accuracy(X[1001:1100,1:],y[1001:1100])
f_time=time.time()
print(acc,f_time-c_time)


ended
0.5151515151515151 0.9671564102172852


In [0]:
a = []
while(1):
    a.append(‘1’)